# macOSにPython環境を構築する（pyenv＋Anaconda）

macOSにPythonの開発環境を構築しました。データ分析と機械学習に興味があるためです。以下に、環境構築手順をまとめます。なお、構築した環境の維持、削除については、[こちら](https://qiita.com/tanuk1647/items/c73aeb0d308dce800750)にまとめました。

ちなみに、別構成（Python＋pipenv）での構築手順は、[こちら](https://qiita.com/tanuk1647/items/cad362af79f528a5c348)です。私自身は、2018/08/16現在、「Python＋pipenv」で構築した環境を利用しています。

## 要件

- 開発用のPythonは、システムにプリインストールされているPythonと分けてインストールすること
- サンドボックス化された、複数の環境を持てるようにすること


## 構成

- macOS High Sierra
- Homebrew
- pyenv
- Anaconda（3系）
- TensorFlow / Keras


## 変更履歴

- 2018/07/26
    - 初版作成

- 2018/07/28
    - タイトルに構成を明記
    - PDFの図表表示位置を明示的に指定する手順を追加（手順8）
    - 設定ファイルへの追記内容に、コメントを追加（手順8、9）
    - フォントについての説明を追加（手順9の補足）
    - 動作確認用Kerasスクリプトを一部修正（手順11）

- 2018/08/04
    - **手順8の修正対象ファイルを、`ipython_config.py`から`ipython_kernel_config.py`に訂正**

- 2018/08/04
    - 段落のインデントを行う設定手順を追加（手順9）

- 2018/08/05
    - 変更履歴の追加
    - 注意事項に、`-bash: hoge: command not found`発生時の対応手順を追加
    - 最新版のAnacondaをインストールするように明記（手順5）
    - **コマンド`conda`を機能拡張するための、`~/.bashrc`への設定手順を追加（手順5）**
    - 仮想環境作成時、ベース環境をクローンして作成する手順に変更（手順7）
    - 仮想環境作成後、`conda update --all`する手順を追加（手順7）
    - **TensorFlow、Kerasを、`pip`でなく`conda`でインストールする手順に変更（手順7）**
    - コマンド`source activate`を`conda activate`に変更（手順7、8）
    - 全般に、見出しのレベルを変更

- 2018/08/05
    - 冒頭に維持管理手順へのリンクを追加
    - 補足追記（手順7）

- 2018/08/13
    - 補足、参考サイト追記（手順6）

- 2018/08/16
    - 冒頭に別構成での環境構築手順へのリンクを追加


## 概要

以下手順で環境構築します。

1. Bash設定ファイルの整備
1. Xcode、Command Line Toolsのインストール
1. Homebrewのインストール
1. pyenvのインストール
1. Anacondaのインストール
1. ワークスペースの作成
1. 仮想環境の作成
1. Jupyter Notebookの設定
1. 日本語まわりの設定
1. GNU Core Utilitiesのインストール
1. 動作確認


## 注意事項

**注意事項**

- コマンド内に`<作成したワークスペース>`等の記載がある場合、各自の環境に合わせて読み替えて実行してください。冗長となるので、特に見落としそうな箇所を除けば、但し書きを書いていません。

```bash
pyenv versions
python -V
cd <作成したワークスペース>
pyenv versions
python -V
pyenv local anaconda<バージョン>
pyenv versions
python -V
```

- コマンドは、一行ずつ実行してください。
- 何かをインストール、設定したはずなのに、`-bash: hoge: command not found`と言って怒られるときは、コマンド`exec $SHELL -l`で、シェルを再起動させると大体うまくいきます。


**参考サイト**

- [シェルを再起動させる簡単な方法](https://qiita.com/yusabana/items/c4de582c6f85a42817d8)


## 謝辞

環境構築、および、本稿の作成にあたっては、参考サイトに記載のサイトのほかにも、多くのサイトを参考にさせて頂きました。心より御礼申し上げます。

## 手順1. Bash設定ファイルの整備

**目的**

- Bash設定ファイルを作成し、メンテナンスしやすい状態にする

**やること**

- Bash設定ファイルの作成
- Bash設定ファイルの編集

**手順**

- ターミナルを起動し、以下コマンドを実行する。

```bash
touch ~/.bash_profile
touch ~/.bashrc
```

- Finderでホームに移動し、`[command] + [shift] + [.]`で、隠しファイルを表示する。
- `.bash_profile`をエディタ（テキストエディット等）で開き、`.bashrc`を呼び出すよう冒頭に記載する。

```bash
test -r ~/.bashrc && . ~/.bashrc
```

**補足**

- 少し単純化して補足説明します。macOSでは、ターミナル起動時にログインシェルが実行され、ホームディレクトリ内の`.bash_profile`が読み込まれます。ターミナル上でコマンド`bash`を実行すると、非ログインシェルが実行され、ホームディレクトリ内の`.bashrc`が読み込まれます。このとき、環境変数は実行元から実行先へと引き継がれます。エイリアス等は引き継がれません。

- 以上をふまえると、以下とするのがスマートだと考えられます。
    - 環境変数は、`.bash_profile`のみに記載する
    - エイリアスの定義、シェル関数の定義、コマンドライン補完の設定は、`.bashrc`のみに記載する
    - `.bash_profile`から`.bashrc`を読み込むように記載する

**参考サイト**

- [Basics of OSX .bashrc v .profile v .bash_profile](https://www.anintegratedworld.com/basics-of-osx-bashrc-v-profile-v-bash_profile/)
- [本当に正しい .bashrc と .bash_profile の使ひ分け](https://qiita.com/magicant/items/d3bb7ea1192e63fba850)


## 手順2. Xcode、Command Line Toolsのインストール

**目的**

- macOS上でビルドできるようにする

**やること**

- Xcodeのインストール
- Command Line Toolsのインストール

**手順**

- App StoreからXcodeをインストールする。
- ターミナルを起動し、コマンド`sudo xcodebuild -license`を実行する。
- コマンド`xcode-select --install`を実行すると、ダイアログが出るので、インストールを選択する。


## 手順3. Homebrewのインストール

**目的**

- macOSにソフトウェアを簡単にインストールできるようにする

**やること**

- Homebrewのインストール

**手順**

- ターミナルを起動し、[Homebrewの公式サイト](https://brew.sh/index_ja)に記載のコマンド（スクリプト）を実行する。


## 手順4. pyenvのインストール

**目的**

- システムにプリインストールされているPythonとは別に、開発用のPythonをインストールできるようにする

**やること**

- pyenvのインストール
- Homebrewの、pyenvに対する警告に対処

**手順**

- ターミナルを起動し、以下コマンドを実行する。

```bash
brew install pyenv
echo 'export PYENV_ROOT="$HOME/.pyenv"' >> ~/.bash_profile
echo 'export PATH="$PYENV_ROOT/bin:$PATH"' >> ~/.bash_profile
echo -e 'if command -v pyenv 1>/dev/null 2>&1; then\n  eval "$(pyenv init -)"\nfi' >> ~/.bashrc
source ~/.bash_profile
source ~/.bashrc
```

- 以下コマンドを実行する。**ただし、`<ユーザ名>`は、コマンド`whoami`の結果で置き換える。**

```bash
whoami
brew doctor
echo 'alias brew="env PATH=${PATH/\/Users\/<ユーザ名>\/\.pyenv\/shims:/} brew"' >> ~/.bashrc
source ~/.bashrc
brew doctor
```

**参考サイト**

- [GitHub - pyenv-pyenv- Simple Python version management](https://github.com/pyenv/pyenv#homebrew-on-mac-os-x)
- [phpenv入れてる時brew doctorしたら出るWarning消した](https://qiita.com/takc923/items/45386905f70fde9af0e7)
- [【Homebrew】brew doctorのWarning対処方法 (その3) + envコマンドについて](http://www.task-notes.com/entry/20141223/1419324649)

## 手順5. Anacondaのインストール

**目的**

- サンドボックス化された、複数の環境を持てるようにする
- 科学技術計算などを中心とした数多くのモジュールやツールをインストールする

**やること**

- インストール可能なAnacondaのバージョンの確認
- 最新版のAnacondaのインストール
- コマンド`conda`の機能拡張

**手順**

- ターミナルを起動し、以下コマンドを実行する。

```bash
pyenv versions
pyenv install -l | grep anaconda
pyenv install anaconda<バージョン>
pyenv versions
echo '. ~/.pyenv/versions/anaconda<バージョン>/etc/profile.d/conda.sh' >> ~/.bashrc
```

**参考サイト**

- [pyenvとanacondaを共存させる時のactivate衝突問題の回避策3種類](https://qiita.com/y__sama/items/f732bb7bec2bff355b69)
- [Recommended change to enable conda in your shell](https://github.com/conda/conda/blob/a4c4feae404b2b378e106bd25f62cc8be15c768f/CHANGELOG.md#440-2017-12-20)


## 手順6. ワークスペースの作成

**目的**

- 開発物を体系的に管理できるようにする
- カレントディレクトリがワークスペースのとき、開発用のPythonが有効になるようにする

**やること**

- ワークスペース用のディレクトリの作成
- ワークスペースにてpyenvのコマンドを実行

**手順**

- ワークスペース用のディレクトリ（例：`~/private/python/workspace`）を作成する。
- ターミナルを起動し、以下コマンドを実行する。

```bash
pyenv versions
python -V
cd <作成したワークスペース>
pyenv versions
python -V
pyenv local anaconda<バージョン>
pyenv versions
python -V
```

**補足**

- `python -V`と入力する際、間違えて小文字で`python -v`と打つと、突如大量の文字列がターミナル上に表示され、戸惑うかもしれません。これは、結論的に言うと、何も問題ありません。単にpythonを対話モードで、かつverbose（冗長）オプション付きで起動したために起きる現象です。あせらず、`quit()`と入力し、対話モードを抜けてください。抜ける際にも、`clear`だの`destroy`だのと言ってビビらせてくれますが、単に環境を閉じているだけなので、安心してください。

**参考サイト**

- [python -vを実行した時のエラーっぽい標準出力について](https://ja.stackoverflow.com/questions/41096/python-v%E3%82%92%E5%AE%9F%E8%A1%8C%E3%81%97%E3%81%9F%E6%99%82%E3%81%AE%E3%82%A8%E3%83%A9%E3%83%BC%E3%81%A3%E3%81%BD%E3%81%84%E6%A8%99%E6%BA%96%E5%87%BA%E5%8A%9B%E3%81%AB%E3%81%A4%E3%81%84%E3%81%A6)


##  手順7. 仮想環境の作成

**目的**

- ベースとなる環境とは別に、開発や実験用に、特定のパッケージをインストールした仮想環境を作成する

**やること**

- 仮想環境の作成
- TensorFlow、Kerasのインストール

**手順**

- ターミナルを起動し、以下コマンドを実行する。

```bash
cd <ワークスペース>
pyenv versions
conda info -e
conda create -n <仮想環境名> --clone base
pyenv versions
conda info -e
conda activate <仮想環境名>
pyenv versions
conda info -e
conda update --all
conda install -c conda-forge tensorflow
conda install keras
conda deactivate
```

**補足**

- [TensorFlow](https://www.tensorflow.org/install/install_mac#installing_with_anaconda)や[Keras](https://keras.io/ja/#_2)の公式サイトには、`pip`を用いてインストールするように指示がありますが、バージョン管理の一貫性の観点から、`conda`を用いてインストールする手順としました。


## 手順8. Jupyter Notebookの設定

**目的**

- Jupyter Notebookから、仮想環境を選択できるようにする
- グラフをインライン表示できるようにする
- 作成したノートブックをLateX経由でPDFダウンロードできるようにする
- プログラミングやマークダウンを快適に行えるようにする

**やること**

- 仮想環境をJupyter Notebookのカーネルとして追加
- iPythonのプロファイルを作成し、グラフインライン表示されるように設定
- MacTexのインストール
- PDFの図表表示位置を明示的に指定
- 日本語等幅フォント（日本語と英語の幅が2:1のもの）のインストール
- 背景色が黒基調となるようなスタイルシートのダウンロード、編集

**手順**

- ターミナルを起動し、以下コマンドを実行する。

```bash
cd <ワークスペース>
jupyter kernelspec list
conda activate <仮想環境名>
ipython kernel install --user --name=<仮想環境名> --display-name=<Jupyter Notebook上での表示名>
jupyter kernelspec list
conda deactivate
```

- ワークスペース内で、コマンド`ipython profile create`を実行する。作成されたプロファイル（`~/.ipython/profile_default/ipython_kernel_config.py`）の最下行に以下を追加する。

```python
## Setting to display graph inline
c.InteractiveShellApp.exec_lines = ['%matplotlib inline']
```

- コマンド`brew cask install mactex`を実行する。
- `~/.pyenv/versions/anaconda<バージョン>/lib/python3.6/site-packages/nbconvert/templates/latex`直下の`base.tplx`を一箇所編集する。
    - PDFの図表が、次ページでなく、なるべくその場所に表示されるように指定（22行目と23行目の間、`\usepackage{graphicx}`の直下に、`% set default figure placement to htbp`以下の4行を追加）

```TeX
〜中略〜
%===============================================================================
% Abstract overrides
%===============================================================================

((* block header *))
    ((* block docclass *))((* endblock docclass *))
    
    ((* block packages *))
    \usepackage[T1]{fontenc}
    % Nicer default font (+ math font) than Computer Modern for most use cases
    \usepackage{mathpazo}

    % Basic figure setup, for now with no caption control since it's done
    % automatically by Pandoc (which extracts ![](path) syntax from Markdown).
    \usepackage{graphicx}
    % set default figure placement to htbp
    \makeatletter
    \def\fps@figure{htbp}
    \makeatother
	% We will generate all images so they have a width \maxwidth. This means
    % that they will get their normal width if they fit onto the page, but
    % are scaled down if they would overflow the margins.
〜中略〜
```

- [Ricty Diminishedの公式サイト](http://www.rs.tus.ac.jp/yyusa/ricty_diminished.html)から、フォントファイル一式をダウンロードする。Font Bookを起動し、ダウンロードしたフォントファイルを追加する。
- [Oleg Levitsky氏のGitHub Gist](https://gist.github.com/levabd/eb2db79567fe737b8232db046ee12eb3)より`custom.css`をダウンロードし、`~/.jupyter/custom/`直下に保存する。保存したcssを三箇所編集する。
    - プログラミング用フォントの`font-family`に`"Ricty Diminished"`を追加（49行目）
    - プログラミング用フォントの`font-size`を`10pt`に変更（58行目）
    - ツールバーの非表示設定をコメントアウト（231行目）

```css
〜（中略）〜
/* 
  SET THE CODE/MONOSPACE FONT 

  I prefer Bitsream Vera Sans Mono, which is freely downloadable here:
  http://www.dafont.com/bitstream-vera-mono.font
  However, there are plenty of alternatives at http://programmingfonts.org/
  in which case, add your preferred font to the stack below.
*/
〜（中略）〜
div.output pre,
div.output code,
.rendered_html pre,
.rendered_html code,
div.completions select,
div.container pre {
    font-family: "Ricty Diminished", "Bitstream Vera Sans Mono", Monaco, monospace;
}

div.output pre,
div.output code,
.rendered_html pre,
.rendered_html p code,
div.completions select,
div.container pre {
    font-size: 10pt;
}
〜（中略）〜
/* NOTEBOOK AND EDITOR INTERFACE */


/* comment out the following line to unhide the toolbar */

div#maintoolbar {
/*    display: none !important; */
}
〜（中略）〜
```

**補足**

- 当初、[jupyterthemes](https://github.com/dunovank/jupyter-themes)の利用も検討しました。コマンドでテーマを切り替えられる素晴らしいツールですが、プログラミングやマークダウン時に日本語等幅フォント（日本語と英語の幅が2:1のもの）を使えるようにすることがどうしてもできなかったので、別の手段をとることにしました。マークダウン時、日本語を含む表を作成する際に、罫線が揃うようにしたかったためです。スタイルシートの作者である[Oleg Levitsky](https://gist.github.com/levabd)氏、および、その先人達に感謝します。
- お気に入りの日本語等幅フォント（日本語と英語の幅が2:1のもの）があれば、そちらを利用しても構いません。フォントのファミリー名は、Font Bookで確認することができます。
- PDFダウンロード時、図表が次ページに送られる現象を当初どうしても解決できませんでした。突破口となったのは、参考サイトの四つ目、@[cawpea](https://qiita.com/cawpea)氏の投稿でした。氏の投稿を参考に、PandocのLaTeX用テンプレートを生成。Jupyter NotebookのMarkdownをコマンド`pandoc`でテンプレートを用いてLaTeXに変換。通常版とDIFFをとることにより、ようやく必要な設定が見えてきた次第です。深く感謝します。

**参考サイト**

- [Jupyterで複数カーネルを簡単に選択するための設定](https://qiita.com/tomochiii/items/8b937f15c79a0c3eae0e)
- [%matplotlib inlineを毎回書かなくて済む方法](https://qiita.com/noguchi0123/items/2c83e30e274e5a51cb41)
- [Installation — nbconvert 5.3.2.dev0 #installing-tex](https://nbconvert.readthedocs.io/en/latest/install.html#installing-tex)
- [Pandocを使ってMarkdownを整形されたHTMLに変換する](https://qiita.com/cawpea/items/cea1243e106ababd15e7)


## 手順9. 日本語まわりの設定

**目的**

- 作成したノートブックをLateX経由でPDFダウンロードする際、日本語が出力されるようにする
- グラフのタイトルやラベルに、日本語が表示されるようにする

**やること**

- LaTeX関連ファイルの作成、編集
- iPythonのプロファイルの編集、キャッシュのクリア

**手順**

- ターミナルを起動し、以下コマンドを実行する。

```bash
mkdir -p ~/texmf/tex/latex/local
touch ~/texmf/tex/latex/local/jalinebreak.sty
```

- 作成された設定ファイル（`jalinebreak.sty`）に以下（日本語の段落を右端で折り返す設定）を記載する。

```TeX
\XeTeXlinebreaklocale "ja"
\XeTeXlinebreakskip=0pt plus 1pt
\XeTeXlinebreakpenalty=0

\def\<{\@ifstar{\zx@hwback\nobreak}{\zx@hwback\relax}}
\def\zx@hwback#1{\leavevmode#1\hskip-.5em\relax}
```

- `~/.pyenv/versions/anaconda<バージョン>/lib/python3.6/site-packages/nbconvert/templates/latex`直下の`base.tplx`を二箇所編集する。
    - フォントを指定する設定、段落のインデントを行う設定、`jalinebreak.sty`を読み込む設定を追加（16行目と17行目の間、`\usepackage[T1]{fontenc}`の直下に、`% Settings for displaying Japanese`以下の5行を追加）
    - 日本語フォントの指定を追加（175行目と177行目の間、`\begin{document}`の下に、`% Settings for displaying Japanese`以下の4行を追加）

```TeX
〜中略〜
%===============================================================================
% Abstract overrides
%===============================================================================

((* block header *))
    ((* block docclass *))((* endblock docclass *))
    
    ((* block packages *))
    \usepackage[T1]{fontenc}
    % Settings for displaying Japanese
    \usepackage{fontspec}
    \setlength{\parindent}{11pt}
    \usepackage{indentfirst}
    \usepackage{jalinebreak}
    % Nicer default font (+ math font) than Computer Modern for most use cases
〜中略〜
((* block body *))
    \begin{document}

    % Settings for displaying Japanese
    \setmainfont[BoldFont=HiraginoSans-W5]{HiraMinProN-W3}
    \setsansfont{HiraginoSans-W5}
    \setmonofont{RictyDiminished-Regular}
    
    ((* block predoc *))
〜中略〜
```

- iPythonのプロファイル（`~/.ipython/profile_default/ipython_kernel_config.py`）の最下行に以下を追加する。

```python
## Settings for displaying Japanese
c.InlineBackend.rc = {
    'font.size': 11,
    'font.family': 'Osaka',
    'figure.figsize': (6.0, 4.0),
    'savefig.dpi': 80,
    'axes.titlesize': 12
}
c.InlineBackend.figure_formats = set(['png', 'retina'])
```

- キャッシュ（`~/.matplotlib/fontList.json`）があれば、削除する。

**補足**

- `base.tplx`の設定では、mainfontに明朝体、BoldFontにその太字、sansfontにゴシック体、monofontに等幅フォントを指定すると、市販の多くの書籍のような書体になります。お気に入りのフォントがあれば、そちらを利用しても構いません。フォントのPostScript名やファミリー名は、Font Bookで確認することができます。
- 私にとってTeXまわりは未知の領域ですので、参考サイトの一つ目、@[masa-ita](https://qiita.com/masa-ita)氏の投稿に出会わなければ、ドツボにハマるところだったでしょう。御礼申し上げます。

**参考サイト**

- [日本語のJupyter NotebookをPDFとしてダウンロードする](https://qiita.com/masa-ita/items/8d5ebe8afe0d580af184)
- [日本語対応のLaTexで自動字下げされない件(＋字下げしない方法)](https://qiita.com/fantm21/items/9e4512d3af605dae1734)
- [Jupyterで日本語ラベルを使ったグラフを描画する](https://qiita.com/hagino3000/items/1b54acc01483ccd0ac72)


## 手順10. GNU Core Utilitiesのインストール

**目的**

- macOSにプリインストールされているBSD系コマンドを、よりメジャーなGNU/Linux系コマンドに置き換える

**やること**

- GNU Core Utilitiesのインストール

**手順**

- ターミナルを起動し、以下コマンドを実行する。

```bash
brew install coreutils
echo 'export PATH="/usr/local/opt/coreutils/libexec/gnubin:$PATH"' >> ~/.bash_profile
echo 'export MANPATH="/usr/local/opt/coreutils/libexec/gnuman:$MANPATH"' >> ~/.bash_profile
echo "alias ls='ls --color=auto'" >> ~/.bashrc
source ~/.bash_profile
source ~/.bashrc
```

- 同様に、diffutils、colordiff、findutils、grepあたりをインストールしておく。コマンド`brew install <パッケージ名>`でよい。追加すべき`PATH`があれば、インストール後に注意事項（`Caveats`）として表示される。

**参考サイト**

- [テキスト処理のための標準的なコマンド群の OS X への導入手順](https://qiita.com/eumesy/items/3bb39fc783c8d4863c5f)
- [散々既出で今更だけどmacOSデフォルトのgrepがめちゃ遅だった](https://amaya382.hatenablog.jp/entry/2017/12/25/180700)


## 手順11. 動作確認

**目的**

- 正しく環境構築できていることを確認する

**やること**

- Jupyter Notebookの背景色が白でなく黒基調であることの確認
- Jupyter Notebookから仮想環境を選択できることの確認
- Kerasが実行できることの確認
- グラフがインライン表示され、ラベルに日本語が表示されていることの確認
- マークダウンが日本語等幅フォントで行えることの確認
- ノートブックをPDF形式でダウンロードでき、日本語が出力されていることの確認
- GNU Core Utilitiesがインストールできていることの確認

**手順**

- 以下コマンドで、Jupyter Notebookを起動し、背景色が白でなく黒基調であることを確認する。

```bash
cd <ワークスペース>
jupyter notebook
```

- 新規ノートブックを作成し、このとき、作成した仮想環境を選択する。（選択できることを確認する。）

- セルに以下のKerasコードを貼り付け、`[shift] + [return]`で実行する。実行できること、グラフが表示されること、および、グラフのラベルに日本語が表示されていることを確認する。

```python
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.layers import Dense
from keras.models import Model
import matplotlib.pyplot as plt


# 学習データを生成する関数
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])


# 正解ラベルを生成する関数
def fizz_buzz_encode(i):
    if   i % 15 == 0: return np.array([0, 0, 0, 1])
    elif i % 5  == 0: return np.array([0, 0, 1, 0])
    elif i % 3  == 0: return np.array([0, 1, 0, 0])
    else:             return np.array([1, 0, 0, 0])


# 訓練データ
NUM_DIGITS = 10
trX = np.array([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
trY = np.array([fizz_buzz_encode(i) for i in range(101, 2 ** NUM_DIGITS)])

# 検証データ
valX = np.array([binary_encode(i, NUM_DIGITS) for i in
                 range(2 ** NUM_DIGITS, 2 ** NUM_DIGITS + 100)])
valY = np.array([fizz_buzz_encode(i) for i in range(1, 101)])


# モデル
model = Sequential()
model.add(Dense(1000, input_dim=10, activation="relu"))
model.add(Dense(1000, activation="relu"))
model.add(Dense(4, activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=["accuracy"])
hist = model.fit(trX, trY, epochs=100, batch_size=128, validation_data=(valX, valY))


# 学習状況の可視化
acc = hist.history['acc']
val_acc = hist.history['val_acc']
plt.plot(range(100), acc, marker='.', label='訓練データ')
plt.plot(range(100), val_acc, marker='.', label='検証データ')
plt.legend(loc='best', fontsize=10)
plt.grid()
plt.xlabel('エポック')
plt.ylabel('正解率')
plt.show()


# バイナリのfizzbuzz変換
def fizz_buzz(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]


# モデルを1から100の整数に適用してみる
numbers = np.arange(1, 101)
teX = np.transpose(binary_encode(numbers, NUM_DIGITS))
teY = model.predict_classes(teX)
output = np.vectorize(fizz_buzz)(numbers, teY)
print (output)


# 正解
answer = np.array([])
for i in numbers:
    if i % 15 == 0: answer = np.append(answer, "fizzbuzz")
    elif i % 5 == 0: answer = np.append(answer, "buzz")
    elif i % 3 == 0: answer = np.append(answer, "fizz")
    else: answer = np.append(answer, str(i))
# print (answer)


# 正解率
evaluate = np.array(answer == output)
print ("正解率：" + str(np.count_nonzero(evaluate == True) / 100))
```

- 新規にセルを追加し、タイプを`Markdown`に切り替える。以下文字列を貼り付け、罫線が揃っていることを確認する。`[shift] + [return]`で表を描画する。

```
| 日本語と         | 英語が混じっても | 罫線が         | ずれない        |
|:-----------------|-----------------:|:--------------:|:---------------:|
| Even if Japanese | and English mix, | the ruled line | does not shift. |
| 日本語と         | 英語が混じっても | 罫線が         | ずれない        |
| Even if Japanese | and English mix, | the ruled line | does not shift. |
| 日本語と         | 英語が混じっても | 罫線が         | ずれない        |
```

- `File`から、LaTex経由のPDF形式でダウンロードし、ダウンロードできること、日本語が出力されていることを確認する。

- Notebookを保存し、閉じる。ブラウザを閉じる。ターミナルを`[control] + [C]`で停止する。

- 新規にターミナルを起動し、コマンド`ls`を実行する。ホームディレクトリ内のディレクトリが紫色で表示されていることを確認する。コマンド`bash`を実行し、コマンド`ls`を実行する。ホームディレクトリ内のディレクトリが紫色で表示されていることを確認する。コマンド`exit`を二回実行し、ターミナルを閉じる。

**補足**

Kerasのコードは、参考サイトの一つ目で@[cvusk氏](https://qiita.com/cvusk)の書かれたコードに、学習時の検証データによる検証を追加したものです。学習状況を可視化する部分は、参考サイトの二つ目を参考にさせて頂きました。それぞれ御礼申し上げます。

**参考サイト**

- [Kerasでfizzbuzz問題を解いてみる](https://qiita.com/cvusk/items/07659830c41b2c3ff02b)
- [KerasではじめるDeepLearning](https://dev.classmethod.jp/machine-learning/introduction-keras-deeplearning/)
